# Discordant Pairs User Study

A user study to uncover the false positive from discordant pairs

**Important Notes**

For evaluation of false positives, each pair of `<original text, mutant text>` must address these questions: 
* Do you think the two pieces of text should have the same semantic meaning? Do you think the second piece of text are more grammatically incorrect than the first text? 
* For the second question, we may not need humans, we can use the grammar checking tools 

To address this task, we need to make a table in csv/excel format that has these columns `original text, mutant text male and female, is two 2 text has sematic meaning, is second text grammatically incorrect`.

In [1]:
import pandas as pd
import numpy as np
import math

#### Read Mutant Original Data

In [11]:
eval_dir = "gc_imdb"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "gender", "template", "template_id"])
df

,label,mutant,gender,template,template_id
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",1846
1,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
2,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
3,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
4,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
...,...,...,...,...,...
1891808,1,"First, I'm a huge Dódi fan. I grew up knowing ...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891809,1,"First, I'm a huge Iboyka fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891810,1,"First, I'm a huge Zsófia fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891811,1,"First, I'm a huge Ágota fan. I grew up knowing...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128


#### Read Mutant Prediction

In [12]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [13]:
output_dir = "gc_imdb"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

1891813


In [14]:
df["prediction"] = pred

#### Separate the Original Text with Mutant Text

In [15]:
dft = df[df["gender"] == "template"]
df = df[df["gender"] != "template"]

# dft = df[df.index % 311 == 0]
# df = df[df.index % 311 != 0]

## template
dft = dft.reset_index(drop=True)

## mutant
df = df.reset_index(drop=True)

In [25]:
## template only


dft = dft.drop(columns=["mutant", "gender"])

# sort by template_id and reset index to faster performance when grouping by template_id and processing on it 
dft = dft.sort_values(["template_id"])
dft = dft.reset_index(drop=True)
dft.head()

,label,template,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,0,0
1,1,Headlines warn us of the current campaign to ...,1,1
2,1,"Heather Graham is not just a pretty face,she ...",2,1
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1
4,0,I understand that people have different expec...,4,0


In [23]:
# mutant only

df = df.sort_values(["template_id", "gender"])
df = df.reset_index(drop=True)
df.head()

,label,mutant,gender,template,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
1,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
2,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
3,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
4,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0


## Get Number of Discordant Pairs for Each Template

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [26]:
gb = df.groupby("template_id")
gb.count()

,label,mutant,gender,template,prediction
template_id,,,,,
0,310,310,310,310,310
1,310,310,310,310,310
2,310,310,310,310,310
3,310,310,310,310,310
4,310,310,310,310,310
...,...,...,...,...,...
6078,310,310,310,310,310
6079,310,310,310,310,310
6080,310,310,310,310,310


In [107]:
import time, random

start = time.time()

identifier = "gender"

mutant_example = []
mutant_prediction_stat = []
key = []
for i in range(len(gb.size())) :
# for i in range(10) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    me = {} # mutant example
    mp = {} # mutant prediction
    key = []
    for k, v in dict(iter(dc)).items() :
        key.append(k)
        pos_counter = 0 # positive counter
        neg_counter = 0 # negative counter
        idx = random.randint(0, len(v["mutant"].values)-1)
        j = 0
        for m, p in zip(v["mutant"].values, v["prediction"].values) :
            if j == idx :
                me[k] = m
            if p == 1 :
                pos_counter += 1
            else :
                neg_counter += 1
            j += 1
        mp[k] = {"pos": pos_counter, "neg" : neg_counter}
        
    mutant_example.append(me)
    mutant_prediction_stat.append(mp)
                            
end = time.time()
print("Execution time: ", end-start)

Execution time:  9.902809619903564


#### Merge with Original Data

In [108]:
dft["mutant_example"] = mutant_example
dft["mutant_prediction_stat"] = mutant_prediction_stat
dft

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,Fourteen of the funniest minutes on celluloid...,0,0,{'female': ' Fourteen of the funniest minutes ...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
1,1,Headlines warn us of the current campaign to ...,1,1,{'female': ' Headlines warn us of the current ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
2,1,"Heather Graham is not just a pretty face,she ...",2,1,{'female': ' Eugenia is not just a pretty face...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
4,0,I understand that people have different expec...,4,0,{'female': ' I understand that people have dif...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
...,...,...,...,...,...,...,...
6078,1,www.petitiononline.com/19784444/petition.html ...,6078,1,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6080,1,you can't take it realistically. -sheets Zombi...,6080,1,{'female': 'you can't take it realistically. -...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6081,1,you know I've seen a lot of crappy hong kong m...,6081,1,{'female': 'you know I've seen a lot of crappy...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0


#### Calculate the Number of Discordant Pairs

In [109]:
dp = []
for mp in dft["mutant_prediction_stat"].values :
    _dp = 0
    for k1 in key :
        for k2 in key :
            if k1 != k2 :
                _dp += mp[k1]["pos"] * mp[k2]["neg"]
                _dp += mp[k1]["neg"] * mp[k2]["pos"]
    
    double_counting_divider = len(key) * (len(key)-1)
    dp.append(int(_dp/double_counting_divider)) # we must divide the number with the number of key to reduce the double counting
                    

In [110]:
dft["number_of_discordant_pairs"] = dp
dft

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,Fourteen of the funniest minutes on celluloid...,0,0,{'female': ' Fourteen of the funniest minutes ...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
1,1,Headlines warn us of the current campaign to ...,1,1,{'female': ' Headlines warn us of the current ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
2,1,"Heather Graham is not just a pretty face,she ...",2,1,{'female': ' Eugenia is not just a pretty face...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
4,0,I understand that people have different expec...,4,0,{'female': ' I understand that people have dif...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
...,...,...,...,...,...,...,...
6078,1,www.petitiononline.com/19784444/petition.html ...,6078,1,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6080,1,you can't take it realistically. -sheets Zombi...,6080,1,{'female': 'you can't take it realistically. -...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6081,1,you know I've seen a lot of crappy hong kong m...,6081,1,{'female': 'you know I've seen a lot of crappy...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0


### Get Data that Have number of discordant pairs more than one 

In [111]:
d = dft[dft["number_of_discordant_pairs"] > 0]
d

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
51,0,(Warning: Some spoilers ahead.) What an incred...,51,1,{'female': '(Warning: Some spoilers ahead.) Wh...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
90,0,...and boy is the collision deafening. A femal...,90,0,{'female': '...and boy is the collision deafen...,"{'female': {'pos': 3, 'neg': 152}, 'male': {'p...",1955
135,1,"A German freshman, Stefan hitch hikes to Paris...",135,1,"{'female': 'A German freshman, Adriana hitch h...","{'female': {'pos': 122, 'neg': 33}, 'male': {'...",6272
141,1,"A Three Stooges short, this one featuring Shem...",141,1,"{'female': 'A Three Stooges short, this one fe...","{'female': {'pos': 110, 'neg': 45}, 'male': {'...",8600
160,0,A dedicated Russian Scientist dreams of going ...,160,0,{'female': ' Tadeja dreams of going to Mars. S...,"{'female': {'pos': 146, 'neg': 9}, 'male': {'p...",3724
...,...,...,...,...,...,...,...
5990,0,like i'm sure other people have said this guy ...,5990,0,{'female': 'like i'm sure other people have sa...,"{'female': {'pos': 1, 'neg': 154}, 'male': {'p...",308
6009,0,radio is possibly one of the best films i have...,6009,1,{'female': 'radio is possibly one of the best ...,"{'female': {'pos': 58, 'neg': 97}, 'male': {'p...",12890
6012,1,so. i was completely in love with this movie. ...,6012,1,{'female': 'so. i was completely in love with ...,"{'female': {'pos': 74, 'neg': 81}, 'male': {'p...",11750
6041,0,this is the worst movie i have ever seen in my...,6041,0,{'female': 'this is the worst movie i have eve...,"{'female': {'pos': 5, 'neg': 150}, 'male': {'p...",2080


In [164]:
d.sum()

label                                                                       113
template                      OK, maybe it doesn't deserve an Oscar. Or a Go...
template_id                                                              759142
prediction                                                                  102
number_of_discordant_pairs                                               982566
dtype: object

#### Sort Data based on the number of discordant pairs

In [112]:
d = d.sort_values(["number_of_discordant_pairs", "template_id"], ascending=False)
d = d.reset_index(drop=True)
d

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,"OK, maybe it doesn't deserve an Oscar. Or a Go...",3495,1,"{'female': 'OK, maybe it doesn't deserve an Os...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",24025
1,0,"I find it terribly ironic that ""left wing"" Hol...",1681,0,"{'female': 'I find it terribly ironic that ""le...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",24025
2,0,I agree with one commentator who says that it'...,1471,1,{'female': 'I agree with one commentator who s...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
3,1,"Excuse me if I'm wrong, but ""Cronica para un d...",1064,0,"{'female': 'Excuse me if I'm wrong, but ""Croni...","{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
4,0,(Warning: Some spoilers ahead.) What an incred...,51,1,{'female': '(Warning: Some spoilers ahead.) Wh...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
...,...,...,...,...,...,...,...
228,0,I did not think Haggard was the funniest movie...,1623,0,{'female': 'I did not think Haggard was the fu...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
229,0,I am a chess player and I wanted to like this ...,1496,0,{'female': 'I am a chess player and I wanted t...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
230,0,How to take Charles Darwin's fantastic intelle...,1433,0,{'female': 'How to take Jandaíra fantastic int...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
231,1,"Hilary was great as julie, and Pat was once ag...",1393,1,"{'female': 'Hilary was great as julie, and Pat...","{'female': {'pos': 154, 'neg': 1}, 'male': {'p...",155


#### Inspect the Example of Mutant tha have a High Number Discordant Pairs

In [113]:
id = [3495, 1681, 1471, 1064, 51]

def print_mutant_example_from_template(i, bug) :
    bug_from_id = bug[bug["template_id"] == i]
    bug_from_id = bug_from_id.iloc[0]
    print()
    print("Template")
    print(bug_from_id["template"])
    print("Male Mutant")
    print(bug_from_id["mutant_example"]["male"])
    print("Female Mutant")
    print(bug_from_id["mutant_example"]["female"])
    print("Number of Discordant Pairs")
    print(bug_from_id["number_of_discordant_pairs"])

# for i in id :
#     print_mutant_example_from_template(i, d)

#### Inspect the Example of Mutant tha have a Low Number Discordant Pairs

In [114]:
id = [1623, 1496, 1433, 1393, 228]

# for i in id :
#     print_mutant_example_from_template(i, d)

#### Create Data for Labelling

In [120]:
id = []
text = []
gender = []
is_make_sense = []
sentiment = []
comment = []

for index, rows in d.iterrows() :
    for i in range(3) :
        id.append(rows["template_id"])
        is_make_sense.append("")
        sentiment.append("")
        comment.append("")
        
    gender.append("t")
    text.append(rows["template"])
    gender.append("m")
    text.append(rows["mutant_example"]["male"])
    gender.append("f")
    text.append(rows["mutant_example"]["female"])

In [121]:
unlabelled = pd.DataFrame(data={"template_id": id, 
                                "gender": gender, 
                                "text": text, 
                                "is_make_sense": is_make_sense, 
                                "sentiment": sentiment,
                                "comment" : comment
                               })

In [122]:
unlabelled

,template_id,gender,text,is_make_sense,sentiment,comment
0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",,,
1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",,,
2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",,,
3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",,,
4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",,,
...,...,...,...,...,...,...
694,1393,m,"Hilary was great as julie, and Pat was once ag...",,,
695,1393,f,"Hilary was great as julie, and Pat was once ag...",,,
696,228,t,A party-hardy frat boy's sister is brutally mu...,,,
697,228,m,A party-hardy frat boy's sister is brutally mu...,,,


In [118]:
unlabelled_shuffled = unlabelled.sample(frac=1, random_state=123456)
unlabelled_shuffled

,template_id,gender,text,is_make_sense,sentiment
566,4717,f,This is a brilliant film. The story starts off...,,
449,2086,f,"I rarely give ratings less than 5, but in this...",,
375,5524,t,"Well, I have to say, this movie was probably t...",,
434,4280,f,The first season told pretty much how all elem...,,
609,5126,t,This movie was Jerry Bruckheimer's idea to sel...,,
...,...,...,...,...,...
171,4998,t,This movie has some of the worst acting I've e...,,
568,4689,m,This game is from a genre of games that tried ...,,
49,1712,m,I found Dominion to be a lousy attempt to cont...,,
498,4670,t,This film was so bad it became enjoyable. If y...,,


In [119]:
unlabelled_shuffled.sample(frac=1, random_state=123456).to_csv("unlabelled.csv")

In [91]:
dft[dft["number_of_discordant_pairs"] == 0]

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,Fourteen of the funniest minutes on celluloid...,0,0,{'female': ' Fourteen of the funniest minutes ...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
1,1,Headlines warn us of the current campaign to ...,1,1,{'female': ' Headlines warn us of the current ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
2,1,"Heather Graham is not just a pretty face,she ...",2,1,{'female': ' Sárinka is not just a pretty face...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
4,0,I understand that people have different expec...,4,0,{'female': ' I understand that people have dif...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
...,...,...,...,...,...,...,...
6078,1,www.petitiononline.com/19784444/petition.html ...,6078,1,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6080,1,you can't take it realistically. -sheets Zombi...,6080,1,{'female': 'you can't take it realistically. -...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6081,1,you know I've seen a lot of crappy hong kong m...,6081,1,{'female': 'you know I've seen a lot of crappy...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0


In [93]:
no_bug = dft[dft["number_of_discordant_pairs"] == 0]

In [105]:
def is_odd(text) :
    return "king" in text

no_bug[no_bug["template"].apply(is_odd)]

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
5,0,"Man, I just cant believe this movie. I have w...",5,0,"{'female': ' Man, I just cant believe this mov...","{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
6,0,Never ever take a film just for its good look...,6,0,{'female': ' Never ever take a film just for i...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
7,1,Oh my word!! I have never seen a film so lack...,7,1,{'female': ' Oh my word!! I have never seen a ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
14,1,When this film was released I dismissed as be...,14,1,{'female': ' When this film was released I dis...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
...,...,...,...,...,...,...,...
6072,0,"who's responsible for these ""behind the scenes...",6072,0,"{'female': 'who's responsible for these ""behin...","{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
6073,0,with this film being directed by Roger Avery a...,6073,0,{'female': 'with this film being directed by S...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
6076,1,wow is all i could say i really loved the movi...,6076,1,{'female': 'wow is all i could say i really lo...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0


In [123]:
# no_bug[no_bug["template"].apply(is_odd)]["mutant_example"].values

### Labelling Result

In [165]:
labelled_result = pd.read_csv("../data/user_study/labelled-full.csv", encoding = "ISO-8859-1")
labelled_result

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
0,568,0,3495,t,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
2,152,2,3495,f,"OK, maybe it doesn't deserve an Oscar. Or a Go...",Yes,Positive,NaN
3,540,3,1681,t,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
4,3,4,1681,m,"I find it terribly ironic that ""left wing"" Hol...",Yes,Negative,NaN
...,...,...,...,...,...,...,...,...
694,147,694,1393,m,"Hilary was great as julie, and Pat was once ag...",Yes,Positive,NaN
695,250,695,1393,f,"Hilary was great as julie, and Pat was once ag...",No,Positive,"Mr. Miyagi, Lotte was ""her"" supposed to be his"
696,554,696,228,t,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN
697,698,697,228,m,A party-hardy frat boy's sister is brutally mu...,Yes,Negative,NaN


In [166]:
male = []
female = []


i = 0
for index, row in labelled_result.iterrows() :
    if i % 3 == 1 :
        male.append(row["sentiment"])
    if i % 3 == 2 :
        female.append(row["sentiment"])
    i += 1 

In [167]:
def lowercasing(text) :
    return text.lower()

dp = pd.DataFrame(data={"male": male, "female": female})
dp["male"] = dp["male"].apply(lowercasing)
dp["female"] = dp["female"].apply(lowercasing)

In [168]:
dp

,male,female
0,positive,positive
1,negative,negative
2,negative,negative
3,positive,positive
4,positive,positive
...,...,...
228,positive,positive
229,negative,negative
230,positive,positive
231,positive,positive


In [169]:
dp[dp["male"] != dp["female"]]

,male,female


In [172]:
# len(labelled_result[labelled_result["is_make_sense"] == "No"])
labelled_result[labelled_result["is_make_sense"] == "No"]

,random_index,sorted_index,template_id,gender,text,is_make_sense,sentiment,comment
1,483,1,3495,m,"OK, maybe it doesn't deserve an Oscar. Or a Go...",No,Positive,Male do strip-tease
14,82,14,51,f,(Warning: Some spoilers ahead.) What an incred...,No,Positive,her pregnant wife
17,271,17,2957,f,It's fun to watch Eugenia doing her thing. She...,No,Positive,even when rejected by her wife to be
19,98,19,664,m,"Before going any further, I have to admit that...",No,Positive,"Hilel made a career of being a happy homemaker,"
44,610,44,3922,f,"Snap, crackle, pop! The jarring sound of every...",No,Positive,laid-back young man -> ÐÑÐ»Ð¸ÑÑÐ° is woman
58,569,58,4331,m,The movie is a starter to what really happened...,No,Positive,"Periklis -> not a person name, itÕs a geograph..."
59,60,59,4331,f,The movie is a starter to what really happened...,No,Positive,"Geisje -> not a person name, itÕs a geographic..."
95,450,95,3130,f,Leaving aside the drawbacks and deficiencies o...,No,Positive,"Rama is girl but "" the problem of power of man..."
100,150,100,4190,m,Samu is very far removed from the Paul Kersey...,No,Positive,Samu -> Paul Kersey of the original film
101,544,101,4190,f,Emke is very far removed from the Paul Kersey...,No,Positive,Emke-> Paul Kersey of the original film
